# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [1]:
#Install the dependencies
!pip install dlt[duckdb]

In [2]:
# to do: homework :)
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1
        
limit = 5
generator = square_root_generator(limit)
sum_of_outputs = sum(generator)
print(sum_of_outputs)
limit = 13
generator = square_root_generator(limit)
for ex in range(12):  # Iterate 12 times to reach the 13th number
    next(generator)
thirteenth_number = next(generator)
print(thirteenth_number)

8.382332347441762
3.605551275463989


In [3]:
import dlt
import duckdb

# Define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# Define your generators
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Load the first generator to the "people_data" table
info_people_1 = generators_pipeline.run(people_1(), table_name="people_data", write_disposition="replace")

# Append the second generator to the same table
info_people_2 = generators_pipeline.run(people_2(), table_name="people_data", write_disposition="append")

# Show outcome
print(info_people_1)
print(info_people_2)

# Connect to DuckDB
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# Set the search path
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

# Calculate the sum of ages for all people
sum_of_ages = conn.execute("SELECT SUM(Age) FROM people_data").fetchone()[0]
print("Sum of ages (Total):", sum_of_ages)

# Display data from "people_data" table
print("\n\n\n people_data table below:")
people_data = conn.sql("SELECT * FROM people_data").df()
display(people_data)

Pipeline dlt_ipykernel_launcher load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Notebooks/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707406265.4078681 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.29 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Notebooks/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707406266.1788528 is LOADED and contains no failed jobs
Sum of ages (Total): 353



 people_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707406265.4078681,dKbqT3ZUAByf/g,None
1,2,Person_2,27,City_A,1707406265.4078681,Q4nD0Wb1IfLyPw,None
2,3,Person_3,28,City_A,1707406265.4078681,wPI/s9UbgrkFmA,None
3,4,Person_4,29,City_A,1707406265.4078681,erJPpg7nEsVkRg,None
4,5,Person_5,30,City_A,1707406265.4078681,Y6JJhJNcY0Gvhw,None
5,3,Person_3,33,City_B,1707406266.1788528,8JMazdKS11NzNg,Job_3
6,4,Person_4,34,City_B,1707406266.1788528,8LKK9AV83K++ow,Job_4
7,5,Person_5,35,City_B,1707406266.1788528,AR/J1JUrOqaSfg,Job_5
8,6,Person_6,36,City_B,1707406266.1788528,7XEfDGRGwNrLGw,Job_6
9,7,Person_7,37,City_B,1707406266.1788528,NxdZJeqLVPwYTQ,Job_7


In [8]:
import dlt
import duckdb

# Define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='merged_generators')

# Define your generators
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Load the first generator to the "people_data" table with a primary key
info_people_1 = generators_pipeline.run(people_1(), table_name="people_data", write_disposition="replace", primary_key="ID")

# Load the second generator and merge it into the existing table
info_people_2 = generators_pipeline.run(people_2(), table_name="people_data", write_disposition="merge", primary_key="ID")

# Show outcome
print(info_people_1)
print(info_people_2)

# Connect to DuckDB
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# Set the search path
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

# Calculate the sum of ages for all people
sum_of_ages = conn.execute("SELECT SUM(Age) FROM people_data").fetchone()[0]
print("Sum of ages (Total):", sum_of_ages)

# Display data from "people_data" table
print("\n\n\n people_data table below:")
people_data = conn.sql("SELECT * FROM people_data").df()
display(people_data)


Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset merged_generators
The duckdb destination used duckdb:////Notebooks/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707406605.1604733 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset merged_generators
The duckdb destination used duckdb:////Notebooks/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707406605.7062638 is LOADED and contains no failed jobs
Sum of ages (Total): 266



 people_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707406605.1604733,o4Xsa0AIC0ZdAQ,None
1,2,Person_2,27,City_A,1707406605.1604733,0LfYcuuu2wmrGw,None
2,5,Person_5,35,City_B,1707406605.7062638,yFufm3Rxw/BI0g,Job_5
3,7,Person_7,37,City_B,1707406605.7062638,fJ8f0C5idNxgcA,Job_7
4,3,Person_3,33,City_B,1707406605.7062638,GIshY7HP3K9PoA,Job_3
5,6,Person_6,36,City_B,1707406605.7062638,geNadFPem4bBDg,Job_6
6,4,Person_4,34,City_B,1707406605.7062638,QgGIOWZYnRO2XA,Job_4
7,8,Person_8,38,City_B,1707406605.7062638,gVZ3crktQ8e5og,Job_8


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX